# Practica 4 (De nou)

## Codi per entrenar models word2vec

In [14]:
from datasets import load_dataset

dataset = load_dataset("projecte-aina/catalan_general_crawling")
dataset = dataset['train']

c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for projecte-aina/catalan_general_crawling contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/projecte-aina/catalan_general_crawling
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [15]:
# Getting the first 100 MB of the dataset
size = 0
for i in range(len(dataset)):
    size += len(dataset[i]['text'])
    if size > 100000000:  # 100 MB
        print(i)
        break

dataset_100MB = dataset.select(list(range(i)))

# Getting the first 500 MB of the dataset
size = 0
for j in range(len(dataset)):
    size += len(dataset[j]['text'])  # corrected from dataset[i]['text']
    if size > 500000000:  # 500 MB
        print(j)
        break

dataset_500MB = dataset.select(list(range(j)))

# Getting the first 1 GB of the dataset
size = 0
for g in range(len(dataset)):
    size += len(dataset[g]['text'])  # corrected from dataset[i]['text']
    if size > 1000000000:  # 1 GB
        print(g)
        break

dataset_1GB = dataset.select(list(range(g)))

# Getting the whole dataset
dataset_full = dataset

57333
213432
410702


In [10]:
import gensim
from gensim.models import Word2Vec
import re

stopwords_catala = [ 
    "a", "abans", "ací", "així", "alguns", "alguna", "algunes", "algú", "alhora", 
    "als", "allò", "aquell", "aquelles", "aquell", "aquells", "baix", 
    "cada", "com", "com a", "eixa", "eixes", "eixí", 
    "eixos", "el", "ella", "elles", "ell", "ells", "en", "endavant", "enfront", 
    "ens", "entre", "he", "hem", "heu", "hi", "ho", 
    "i", "igual", "iguals", "ja", "jo", "l'", "la", "les", "li", "els", "tu", "nosaltres", "vosaltres",
    "de", "del", "dels", "d'un", "d'una", "d'uns", "d'unes", "des", "des de",
    ]

# Funció per netejar i tokenitzar el text
def preprocess_text(text):
    # Neteja el text: treu caràcters no desitjats, converteix a minúscules, etc.
    text = re.sub(r'\W+', ' ', text)  # Substitueix caràcters no alfanumèrics per espais
    text = text.lower()  # Converteix a minúscules
    words = text.split()  # Tokenitza
    # elimina stopwords
    words = [word for word in words if word not in stopwords_catala]
    return words

# Carrega el fitxer de text i processa'l
corpus_100MB = []
for line in dataset_100MB:
    words = preprocess_text(line['text'])
    if words:  # Assegura't que la línia no està buida
        corpus_100MB.append(words)

# Comprova algunes línies per assegurar-te que tot està bé
print(corpus_100MB[:2])

NameError: name 'dataset_100MB' is not defined

In [ ]:
import gensim
from gensim.models import Word2Vec
from gensim.models.word2vec import PathLineSentences
import os

def entrenar_word2vec(directori_corpus, mida_vector=100, finestra=5, sg=1):
    sentences = PathLineSentences(directori_corpus)
    model = Word2Vec(sentences, vector_size=mida_vector, window=finestra, sg=sg)
    return model

# Entrenar models per a cada mida de dataset
models = {}
for mida in ["100MB", "500MB", "1GB", "complet"]:
    directori_corpus = f"dataset_{mida}"
    model = entrenar_word2vec(directori_corpus)
    models[mida] = model
    model.save(f"word2vec_skipgram_{mida}.model")


## Model Baseline

In [ ]:
import tensorflow as tf

def build_and_compile_model(input_length: int = 764, hidden_size: int = 64, dictionary_size: int = 1000, embedding_size: int = 16) -> tf.keras.Model:
    
    input_1, input_2 = tf.keras.Input((input_length, ), dtype=tf.int32), tf.keras.Input((input_length, ), dtype=tf.int32)
    
    # Define Layers
    embedding = tf.keras.layers.Embedding(
        dictionary_size, embedding_size, input_length=input_length, mask_zero=True)
    
    lstm = tf.keras.layers.LSTM(hidden_size)
    
    concatenate = tf.keras.layers.Concatenate(axis=-1)
    hidden = tf.keras.layers.Dense(hidden_size, activation='relu')
    output = tf.keras.layers.Dense(1)
    
    # Pass through the layers
    _embedded_1, _embedded_2 = embedding(input_1), embedding(input_2)
    _lstm_1, _lstm_2 = lstm(_embedded_1), lstm(_embedded_2)
    
    _concatenated = concatenate([_lstm_1, _lstm_2])
    _hidden_output = hidden(_concatenated)
    _output = output(_hidden_output)
    
    # Define the model
    model = tf.keras.Model(inputs=(input_1, input_2), outputs=_output)
    model.compile(loss='mean_absolute_error',
                  optimizer=tf.keras.optimizers.Adam(0.01))
    return model

# Exemple d'ús del model
baseline_model = build_and_compile_model()
baseline_model.summary()

## Model de Similitud de Text Semàntic

In [6]:
# Requisites
from gensim.models import TfidfModel
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
import numpy as np
from scipy import spatial
from typing import Tuple, List

In [1]:
from datasets import load_dataset

text_semantic = load_dataset("projecte-aina/sts-ca")

c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for projecte-aina/sts-ca contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/projecte-aina/sts-ca
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [5]:
text_semantic['train']['sentence1'][0]

'Atorga per primer cop les mencions Encarna Sanahuja a la inclusió de la perspectiva de gènere en docència Universitària'

In [7]:
input_pairs = [(e["sentence1"], e["sentence2"], e["label"], ) for e in text_semantic["train"].to_list()]
input_pairs_val = [(e["sentence1"], e["sentence2"], e["label"], ) for e in text_semantic["validation"].to_list()]
input_pairs_test = [(e["sentence1"], e["sentence2"], e["label"], ) for e in text_semantic["test"].to_list()]

### One Hot

In [ ]:
# Preprcessament del text_semantic
corpus_semantic = []
semantic_score = []
for line in text_semantic['train']:
    frase1 = preprocess_text(line['sentence1'])
    frase2 = preprocess_text(line['sentence2'])

    corpus_semantic.append((frase1, frase2))
    semantic_score.append(line['label'])

In [ ]:
corpus_semantic_test = []
semantic_score_test = []
for line in text_semantic['test']:
    frase1 = preprocess_text(line['sentence1'])
    frase2 = preprocess_text(line['sentence2'])

    corpus_semantic_test.append((frase1, frase2))
    semantic_score_test.append(line['label'])

In [ ]:
vocabulari = {}

for frases in corpus_semantic:
    for frase in frases:
        for paraula in frase:
            if paraula in vocabulari:
                vocabulari[paraula] += 1
            else:
                vocabulari[paraula] = 1

In [ ]:
vocabulari_reduit = {palabra for palabra, frecuencia in vocabulari.items() if frecuencia >= 10}

In [ ]:
import numpy as np

# Función para calcular la representación one-hot de un texto
def compute_one_hot_encoding(text, vocabulari):
    """
    Genera la representación one-hot de un texto utilizando un vocabulario predefinido.
    """
    one_hot_vector = np.zeros(len(vocabulari))
    vocabulario_list = list(vocabulari)
    for palabra in text:
        if palabra in vocabulari:
            index = vocabulario_list.index(palabra)
            one_hot_vector[index] = 1
        
    return one_hot_vector

In [ ]:
# Passar les frases de corpus_semantic a vectors
one_hot_corpus_semantic = corpus_semantic.copy()
for i, (frase1, frase2) in enumerate(one_hot_corpus_semantic):
    one_hot_corpus_semantic[i] = (compute_one_hot_encoding(frase1, vocabulari_reduit), 
                                    compute_one_hot_encoding(frase2, vocabulari_reduit))

In [ ]:
#### Entrenament de la xarxa amb els vectors de les frases ####

# Convertir les llistes de tuples a arrays de NumPy
vectors_1 = np.array([pair[0] for pair in one_hot_corpus_semantic])
vectors_2 = np.array([pair[1] for pair in one_hot_corpus_semantic])

# Comprovar si hi ha valors fora de rang i substituir-los
vectors_1 = np.clip(vectors_1, 0, 1)
vectors_2 = np.clip(vectors_2, 0, 1)

# Convertir les llistes de similituds a un array de NumPy
semantic_score = np.array(semantic_score)

baseline_model.fit(
    [vectors_1, vectors_2], 
    semantic_score, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Pasar las frases de corpus_semantic a vectores
one_hot_corpus_semantic_test = corpus_semantic_test.copy()
for i, (frase1, frase2) in enumerate(one_hot_corpus_semantic_test):
    one_hot_corpus_semantic_test[i] = (compute_one_hot_encoding(frase1, vocabulari_reduit), 
                                    compute_one_hot_encoding(frase2, vocabulari_reduit))

vectors_1 = np.array([pair[0] for pair in one_hot_corpus_semantic_test])
vectors_2 = np.array([pair[1] for pair in one_hot_corpus_semantic_test])

# Obtener predicciones del modelo
predicciones = baseline_model.predict([vectors_1, vectors_2])

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(semantic_score_test, predicciones)

# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(semantic_score_test, predicciones)

# Mostrar las predicciones, el MSE y el R^2
#print("Predicciones:", predicciones)
print("Error cuadrático medio (MSE):", mse)
print("Coeficiente de determinación (R^2):", r2)

### Model Word2Vec

Realitzem el model de regressio amb els embeddings dels models Word2Vec + Mean i + Mean Ponderada.

In [ ]:
all_input_pairs = input_pairs + input_pairs_val + input_pairs_test
# Preprocesamiento de las oraciones y creación del diccionario
sentences_1_preproc = [simple_preprocess(sentence_1) for sentence_1, _, _ in all_input_pairs]
sentences_2_preproc = [simple_preprocess(sentence_2) for _, sentence_2, _ in all_input_pairs]
sentence_pairs = list(zip(sentences_1_preproc, sentences_2_preproc))
# Versión aplanada para poder entrenar el modelo
sentences_pairs_flattened = sentences_1_preproc + sentences_2_preproc
diccionario = Dictionary(sentences_pairs_flattened)

In [ ]:
# Cálculo de los pesos TF-IDF para las oraciones pre-procesadas
corpus = [diccionario.doc2bow(sent) for sent in sentences_pairs_flattened]
modelo_tfidf = TfidfModel(corpus)

In [ ]:
def map_tf_idf(sentence_preproc: List[str], dictionary: Dictionary, tf_idf_model: TfidfModel) -> Tuple[List[np.ndarray], List[float]]:
    bow = dictionary.doc2bow(sentence_preproc)
    tf_idf = tf_idf_model[bow]
    vectors, weights = [], []
    for word_index, weight in tf_idf:
        word = dictionary.get(word_index)
        if word in wv_model:
            vectors.append(wv_model[word])
            weights.append(weight)
    return vectors, weights

def map_pairs(
        sentence_pairs: List[Tuple[str, str, float]],
        dictionary: Dictionary = None,
        tf_idf_model: TfidfModel = None,
) -> List[Tuple[Tuple[np.ndarray, np.ndarray], float]]:
    # Mapeo de los pares de oraciones a pares de vectores
    pares_vectores = []
    for i, (sentence_1, sentence_2, similitud) in enumerate(sentence_pairs):
        sentence_1_preproc = preprocess_text(sentence_1)
        sentence_2_preproc = preprocess_text(sentence_2)
        # Si usamos TF-IDF
        if tf_idf_model is not None:
            # Cálculo del promedio ponderado por TF-IDF de los word embeddings
            vectors1, weights1 = map_tf_idf(sentence_1_preproc, dictionary=dictionary, tf_idf_model=tf_idf_model, )
            vectors2, weights2 = map_tf_idf(sentence_2_preproc, dictionary=dictionary, tf_idf_model=tf_idf_model, )
            vector1 = np.average(vectors1, weights=weights1, axis=0, )
            vector2 = np.average(vectors2, weights=weights2, axis=0, )
        else:
            # Cálculo del promedio de los word embeddings
            vectors1 = [wv_model[word] for word in sentence_1_preproc if word in wv_model]
            vectors2 = [wv_model[word] for word in sentence_2_preproc if word in wv_model]
            vector1 = np.mean(vectors1, axis=0)
            vector2 = np.mean(vectors2, axis=0)
        # Añadir a la lista
        pares_vectores.append(((vector1, vector2), similitud))
    return pares_vectores

### Word2Vec + Mean Ponderada

In [ ]:
# Imprimir los pares de vectores y la puntuación de similitud asociada
mapped = map_pairs(input_pairs, tf_idf_model=modelo_tfidf, dictionary=diccionario, )
# Imprimir los pares de vectores y la puntuación de similitud asociada
mapped_train = map_pairs(input_pairs,  tf_idf_model=modelo_tfidf, dictionary=diccionario, )
mapped_val = map_pairs(input_pairs_val, tf_idf_model=modelo_tfidf, dictionary=diccionario, )
mapped_test = map_pairs(input_pairs_test, tf_idf_model=modelo_tfidf, dictionary=diccionario, )

### Word2Vec + Mean amb model de Linear Regression

In [19]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Exemple de preprocessament de text
def obtenir_vectores_mean(text, model_word2vec):
    tokens = text.split()
    vectors = [model_word2vec.wv[word] for word in tokens if word in model_word2vec.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model_word2vec.vector_size)

# Exemple d'entrenament del model de regressió
def entrenar_model_regressio(texts, labels, model_word2vec):
    X = np.array([np.concatenate((obtenir_vectores_mean(text1, model_word2vec), obtenir_vectores_mean(text2, model_word2vec))) 
                  for text1, text2 in texts])
    
    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2)
    model = LinearRegression()
    model.fit(X_train, y_train)
    prediccions = model.predict(X_test)
    mse = mean_squared_error(y_test, prediccions)
    return model, mse

# Entrenar el model de regressió utilitzant un dels models Word2Vec
texts =  [(sentence1, sentence2) for sentence1, sentence2, _ in input_pairs] # textos del dataset
labels = [label for _, _, label in input_pairs]  # similituds reals
model_word2vec = Word2Vec.load("model_100MB") # utilitzar el model Word2Vec entrenat

model_regressio, mse = entrenar_model_regressio(texts, labels, model_word2vec)
print(f"MSE del model de regressió: {mse}")


MSE del model de regressió: 0.6825629832324949


Un valor d'MSE de 0.6825629832324949 indica que l'error quadràtic mitjà entre les prediccions del model i els valors reals és relativament alt. Això suggereix que el model de regressió no està fent prediccions molt precises.

In [20]:
# Funció per predir la similitud entre dos textos
def predir_similitud(text1, text2, model_regressio, model_word2vec):
    vector1 = obtenir_vectores_mean(text1, model_word2vec)
    vector2 = obtenir_vectores_mean(text2, model_word2vec)
    vector_concat = np.concatenate((vector1, vector2)).reshape(1, -1)
    prediccio = model_regressio.predict(vector_concat)
    return prediccio[0]

# Exemple de predicció
text1 = "Aquest és un text"
text2 = "Aquest és un altre text"
similitud = predir_similitud(text1, text2, model_regressio, model_word2vec)
print(f"Similitud predita: {similitud}")


Similitud predita: 2.6517136096954346


### 3. spaCy

In [ ]:
!py -m spacy download ca_core_news_md

In [ ]:
import spacy

nlp_md = spacy.load('ca_core_news_md')

In [ ]:
def spacy_mean_embedding(text1, text2, model):
    """
    Agafa els vectors d'embeddings de cada paraula en una frase o document i 
    calcula la mitjana dels vectors per obtenir una única representació vectorial 
    per a la frase o document.
    """
    text1 = ' '.join(text1)
    text2 = ' '.join(text2)
    frase1 = model(text1).vector
    frase2 = model(text2).vector
    
    return frase1, frase2

In [ ]:
# Passar les frases de corpus_semantic a vectors
spacy_mean_embbeding_corpus_semantic = corpus_semantic.copy()
for i, (frase1, frase2) in enumerate(spacy_mean_embbeding_corpus_semantic):
    
    frase1, frase2 = spacy_mean_embedding(frase1, frase2, nlp_md)
    spacy_mean_embbeding_corpus_semantic[i] = (frase1, frase2)

In [ ]:
#### Entrenament de la xarxa amb els vectors de les frases ####

# Convertir les llistes de tuples a arrays de NumPy
vectors_1 = np.array([pair[0] for pair in spacy_mean_embbeding_corpus_semantic])
vectors_2 = np.array([pair[1] for pair in spacy_mean_embbeding_corpus_semantic])

# Comprovar si hi ha valors fora de rang i substituir-los
#vectors_1 = np.clip(vectors_1, 0, 999)
#vectors_2 = np.clip(vectors_2, 0, 999)

# Convertir les llistes de similituds a un array de NumPy
semantic_score = np.array(semantic_score)

print(vectors_1.shape)
print(vectors_2.shape)
print(semantic_score.shape)

baseline_model.fit(
    [vectors_1, vectors_2], 
    semantic_score, 
    epochs=50, 
    batch_size=32, 
    

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Pasar las frases de corpus_semantic a vectores
spacy_mean_embbeding_corpus_semantic_test = corpus_semantic_test.copy()
for i, (frase1, frase2) in enumerate(spacy_mean_embbeding_corpus_semantic_test):
    
    frase1, frase2 = spacy_mean_embedding(frase1, frase2, nlp_md)
    spacy_mean_embbeding_corpus_semantic_test[i] = (frase1, frase2)

vectors_1 = np.array([pair[0] for pair in spacy_mean_embbeding_corpus_semantic_test])
vectors_2 = np.array([pair[1] for pair in spacy_mean_embbeding_corpus_semantic_test])

# Obtener predicciones del modelo
predicciones = baseline_model.predict([vectors_1, vectors_2])

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(semantic_score_test, predicciones)

# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(semantic_score_test, predicciones)

# Mostrar las predicciones, el MSE y el R^2
#print("Predicciones:", predicciones)
print("Error cuadrático medio (MSE):", mse)
print("Coeficiente de determinación (R^2):", r2)

### 4.RoBERTa

In [ ]:
!py -m spacy download ca_core_news_trf

In [ ]:
import spacy

nlp_trf = spacy.load('ca_core_news_trf')

In [ ]:
def spacy_trf_CLS(frase1, frase2, model):# Procesar la frase
    frase1 = ' '.join(frase1)
    frase2 = ' '.join(frase2)
    doc1 = model(frase1)
    doc2 = model(frase2)

    # Obtener el embedding CLS
    embedding_CLS1 = doc1._.trf_data.last_hidden_layer_state
    embedding_array1 = embedding_CLS1.data
    embedding_CLS2 = doc2._.trf_data.last_hidden_layer_state
    embedding_array2 = embedding_CLS2.data
    
    return embedding_array1, embedding_array2

In [ ]:
spacy_trf_CLS_corpus_semantic = corpus_semantic.copy()
for i, (frase1, frase2) in enumerate(spacy_trf_CLS_corpus_semantic):
    
    frase1, frase2 = spacy_trf_CLS(frase1, frase2, nlp_trf)
    spacy_trf_CLS_corpus_semantic[i] = (frase1, frase2)

In [ ]:
#### Entrenament de la xarxa amb els vectors de les frases ####

# Convertir les llistes de tuples a arrays de NumPy
vectors_1 = np.array([pair[0][0] for pair in spacy_trf_CLS_corpus_semantic])
vectors_2 = np.array([pair[0][1] for pair in spacy_trf_CLS_corpus_semantic])

# Comprovar si hi ha valors fora de rang i substituir-los
#vectors_1 = np.clip(vectors_1, 0, 999)
#vectors_2 = np.clip(vectors_2, 0, 999)

# Convertir les llistes de similituds a un array de NumPy
semantic_score = np.array(semantic_score)

print(vectors_1.shape)
print(vectors_2.shape)
print(semantic_score.shape)

baseline_model.fit(
    [vectors_1, vectors_2], 
    semantic_score, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Pasar las frases de corpus_semantic a vectores
spacy_trf_CLS_corpus_semantic_test = corpus_semantic_test.copy()
for i, (frase1, frase2) in enumerate(spacy_trf_CLS_corpus_semantic_test):
    
    frase1, frase2 = spacy_trf_CLS(frase1, frase2, nlp_trf)
    spacy_trf_CLS_corpus_semantic_test[i] = (frase1, frase2)

vectors_1 = np.array([pair[0][0] for pair in spacy_trf_CLS_corpus_semantic_test])
vectors_2 = np.array([pair[0][1] for pair in spacy_trf_CLS_corpus_semantic_test])

# Obtener predicciones del modelo
predicciones = baseline_model.predict([vectors_1, vectors_2])

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(semantic_score_test, predicciones)

# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(semantic_score_test, predicciones)

# Mostrar las predicciones, el MSE y el R^2
#print("Predicciones:", predicciones)
print("Error cuadrático medio (MSE):", mse)
print("Coeficiente de determinación (R^2):", r2)

In [ ]:
import spacy
import numpy as np

# Cargar el modelo
nlp_trf = spacy.load('ca_core_news_trf')

# Frase de ejemplo
frase1 = 'atorga per primer cop les mencions encarna sanahuja a la inclusió de la perspectiva de gènere en docència universitària'
doc = nlp_trf(frase1)

# Obtener las representaciones de la última capa oculta
embeddings = doc._.trf_data.last_hidden_layer_state

# Convertir a numpy array
embeddings_np = np.array(embeddings.data)

# Calcular la media de los embeddings a lo largo del eje de las palabras
mean_embedding = np.mean(embeddings_np, axis=1)

print(mean_embedding)

In [ ]:
def spacy_trf_mean(frase1, frase2, model):
    frase1 = ' '.join(frase1)
    frase2 = ' '.join(frase2)
    doc1 = model(frase1)
    doc2 = model(frase2)

    # Obtener el embedding CLS
    embedding_CLS1 = doc1._.trf_data.last_hidden_layer_state
    embedding_array1 = embedding_CLS1.data
    embedding_CLS2 = doc2._.trf_data.last_hidden_layer_state
    embedding_array2 = embedding_CLS2.data
    
    return np.array(embedding_array1.data), np.array(embedding_array2.data)

In [ ]:
spacy_trf_mean_corpus_semantic = corpus_semantic.copy()
for i, (frase1, frase2) in enumerate(spacy_trf_mean_corpus_semantic):
    
    frase1, frase2 = spacy_trf_CLS(frase1, frase2, nlp_trf)
    spacy_trf_mean_corpus_semantic[i] = (frase1, frase2)

In [ ]:
#### Entrenament de la xarxa amb els vectors de les frases ####

# Convertir les llistes de tuples a arrays de NumPy
vectors_1 = np.array([pair[0][0] for pair in spacy_trf_CLS_corpus_semantic])
vectors_2 = np.array([pair[0][1] for pair in spacy_trf_CLS_corpus_semantic])

# Comprovar si hi ha valors fora de rang i substituir-los
#vectors_1 = np.clip(vectors_1, 0, 999)
#vectors_2 = np.clip(vectors_2, 0, 999)

# Convertir les llistes de similituds a un array de NumPy
semantic_score = np.array(semantic_score)

print(vectors_1.shape)
print(vectors_2.shape)
print(semantic_score.shape)

baseline_model.fit(
    [vectors_1, vectors_2], 
    semantic_score, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Pasar las frases de corpus_semantic a vectores
spacy_trf_mean_corpus_semantic_test = corpus_semantic_test.copy()
for i, (frase1, frase2) in enumerate(spacy_trf_mean_corpus_semantic_test):
    
    frase1, frase2 = spacy_trf_mean(frase1, frase2, nlp_trf)
    spacy_trf_mean_corpus_semantic_test[i] = (frase1, frase2)

vectors_1 = np.array([pair[0][0] for pair in spacy_trf_mean_corpus_semantic_test])
vectors_2 = np.array([pair[0][1] for pair in spacy_trf_mean_corpus_semantic_test])

# Obtener predicciones del modelo
predicciones = baseline_model.predict([vectors_1, vectors_2])

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(semantic_score_test, predicciones)

# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(semantic_score_test, predicciones)

# Mostrar las predicciones, el MSE y el R^2
#print("Predicciones:", predicciones)
print("Error cuadrático medio (MSE):", mse)
print("Coeficiente de determinación (R^2):", r2)

### 5. RoBERTa fine-tuned

In [ ]:
from transformers import pipeline, AutoTokenizer
from scipy.special import logit

model = 'projecte-aina/roberta-base-ca-v2-cased-sts'
tokenizer = AutoTokenizer.from_pretrained(model)
pipe = pipeline('text-classification', model=model, tokenizer=tokenizer)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

model_name = 'projecte-aina/roberta-base-ca-v2-cased-sts'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [ ]:
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state

In [ ]:
RoBERTa_corpus_semantic = corpus_semantic.copy()
for i, (frase1, frase2) in enumerate(RoBERTa_corpus_semantic):
    frase1 = get_embeddings(frase1)
    frase2 = get_embeddings(frase2)
    RoBERTa_corpus_semantic[i] = (frase1, frase2)

In [ ]:
#### Entrenament de la xarxa amb els vectors de les frases ####

# Convertir les llistes de tuples a arrays de NumPy
vectors_1 = np.array([pair[0][0] for pair in RoBERTa_corpus_semantic])
vectors_2 = np.array([pair[0][1] for pair in RoBERTa_corpus_semantic])

# Comprovar si hi ha valors fora de rang i substituir-los
#vectors_1 = np.clip(vectors_1, 0, 999)
#vectors_2 = np.clip(vectors_2, 0, 999)

# Convertir les llistes de similituds a un array de NumPy
semantic_score = np.array(semantic_score)

print(vectors_1.shape)
print(vectors_2.shape)
print(semantic_score.shape)

baseline_model.fit(
    [vectors_1, vectors_2], 
    semantic_score, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

In [ ]:
def prepare(sentence_pairs):
    sentence_pairs_prep = []
    for s1, s2 in sentence_pairs:
        sentence_pairs_prep.append(f"{tokenizer.cls_token} {s1}{tokenizer.sep_token}{tokenizer.sep_token} {s2}{tokenizer.sep_token}")
    return sentence_pairs_prep

In [ ]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
model_name = 'projecte-aina/roberta-base-ca-v2-cased-sts'
tokenizer = AutoTokenizer.from_pretrained(model_name)
transformer_model = AutoModel.from_pretrained(model_name)

# Convert sentence pairs to embeddings using the transformer model
def get_embeddings(sentence_pairs):
    prepared_pairs = prepare(sentence_pairs)
    embeddings = []
    for pair in prepared_pairs:
        inputs = tokenizer(pair, return_tensors='pt', truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = transformer_model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())
    return embeddings

fine_tuned_corpus_semantic = corpus_semantic.copy()
for i, (frase1, frase2) in enumerate(fine_tuned_corpus_semantic):
    fine_tuned_corpus_semantic[i] = (' '.join(frase1), ' '.join(frase2))

embeddings = get_embeddings(fine_tuned_corpus_semantic)

In [ ]:
fine_runed_corpus_semantic = corpus_semantic.copy()
for i, (frase1, frase2) in enumerate(fine_runed_corpus_semantic):
    frase1 = ' '.join(frase1)
    frase2 = ' '.join(frase2)
    fine_runed_corpus_semantic[i] = (frase1, frase2)

In [ ]:
predictions = pipe(prepare(fine_runed_corpus_semantic), add_special_tokens=False)

# convert back to scores to the original 0 and 5 interval
for prediction in predictions:
    prediction['score'] = logit(prediction['score'])
print(predictions)

In [ ]:
def compute_mean_embedding(text, model, embedding_dim):
    """
    Agafa els vectors d'embeddings de cada paraula en una frase o document i 
    calcula la mitjana dels vectors per obtenir una única representació vectorial 
    per a la frase o document.
    """
    vectors = [model[word] for word in text if word in model]
    if vectors:
        mean_vector = np.mean(vectors, axis=0)
    else:
        mean_vector = np.zeros(embedding_dim)
    return mean_vector


from sklearn.metrics import mean_squared_error, r2_score

# Pasar las frases de corpus_semantic a vectores
mean_embbeding_corpus_semantic_test = corpus_semantic_test.copy()
for i, (frase1, frase2) in enumerate(mean_embbeding_corpus_semantic_test):
    mean_embbeding_corpus_semantic_test[i] = (
        compute_mean_embedding(frase1, model_100MB.wv, 100),
        compute_mean_embedding(frase2, model_100MB.wv, 100)
    )

vectors_1 = np.array([pair[0] for pair in mean_embbeding_corpus_semantic_test])
vectors_2 = np.array([pair[1] for pair in mean_embbeding_corpus_semantic_test])

# Obtener predicciones del modelo
predicciones = baseline_model.predict([vectors_1, vectors_2])

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(semantic_score_test, predicciones)

# Calcular el coeficiente de determinación (R^2)
r2 = r2_score(semantic_score_test, predicciones)

# Mostrar las predicciones, el MSE y el R^2
#print("Predicciones:", predicciones)
print("Error cuadrático medio (MSE):", mse)
print("Coeficiente de determinación (R^2):", r2)

### model amb embeddings entrenables

#### Random Embeddings

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec

vectors_1 =[pair[0] for pair in mean_embbeding_corpus_semantic]
vectors_2 =[pair[1] for pair in mean_embbeding_corpus_semantic]

# Tokenització i creació del vocabulari
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts_1 + texts_2)
sequences_1 = tokenizer.texts_to_sequences(texts_1)
sequences_2 = tokenizer.texts_to_sequences(texts_2)
word_index = tokenizer.word_index

# Padding de les seqüències
max_length = 10
data_1 = pad_sequences(sequences_1, maxlen=max_length)
data_2 = pad_sequences(sequences_2, maxlen=max_length)

# Paràmetres del model
vocab_size = len(word_index) + 1
embedding_dim = 100
hidden_size = 64
input_length = max_length

# Creació d'embeddings aleatoris
random_embeddings = np.random.uniform(-1, 1, (vocab_size, embedding_dim))

# Creació d'embeddings de Word2Vec
texts_combined = [text.split() for text in texts_1 + texts_2]
word2vec_model = Word2Vec(texts_combined, vector_size=embedding_dim, window=5, min_count=1, workers=4)
word2vec_embeddings = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    if word in word2vec_model.wv:
        word2vec_embeddings[i] = word2vec_model.wv[word]

# Entrenament del model amb embeddings aleatoris
model_random = build_and_compile_model(random_embeddings, input_length=max_length, hidden_size=hidden_size)
model_random.fit([data_1, data_2], labels, epochs=10, batch_size=2)

# Entrenament del model amb embeddings de Word2Vec
model_word2vec = build_and_compile_model(word2vec_embeddings, input_length=max_length, hidden_size=hidden_size)
model_word2vec.fit([data_1, data_2], labels, epochs=10, batch_size=2)

# Avalua els resultats dels models
loss_random = model_random.evaluate([data_1, data_2], labels)
loss_word2vec = model_word2vec.evaluate([data_1, data_2], labels)

print(f'Random Embeddings - Loss: {loss_random}')
print(f'Word2Vec Embeddings - Loss: {loss_word2vec}')

#### Word2Vec

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec

# Exemple de dades
texts_1 = ["aquest és un text d'exemple", "un altre text"]
texts_2 = ["aquest és un altre text", "un text diferent"]
labels = [0.8, 0.4]  # Similitud entre les frases

# Tokenització i creació del vocabulari
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts_1 + texts_2)
sequences_1 = tokenizer.texts_to_sequences(texts_1)
sequences_2 = tokenizer.texts_to_sequences(texts_2)
word_index = tokenizer.word_index

# Padding de les seqüències
max_length = 10
data_1 = pad_sequences(sequences_1, maxlen=max_length)
data_2 = pad_sequences(sequences_2, maxlen=max_length)

# Paràmetres del model
vocab_size = len(word_index) + 1
embedding_dim = 100
hidden_size = 64
input_length = max_length

# Creació d'embeddings de Word2Vec
texts_combined = [text.split() for text in texts_1 + texts_2]
word2vec_model = Word2Vec(texts_combined, vector_size=embedding_dim, window=5, min_count=1, workers=4)
word2vec_embeddings = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    if word in word2vec_model.wv:
        word2vec_embeddings[i] = word2vec_model.wv[word]

def build_and_compile_model(embedding_matrix, input_length: int = 100, hidden_size: int = 64) -> tf.keras.Model:
    input_1, input_2 = tf.keras.Input((input_length, ), dtype=tf.int32), tf.keras.Input((input_length, ), dtype=tf.int32)

    # Define Layers
    embedding = tf.keras.layers.Embedding(
        vocab_size, embedding_dim, weights=[embedding_matrix], input_length=input_length, mask_zero=True, trainable=True)

    lstm = tf.keras.layers.LSTM(hidden_size)

    concatenate = tf.keras.layers.Concatenate(axis=-1)
    hidden = tf.keras.layers.Dense(hidden_size, activation='relu')
    output = tf.keras.layers.Dense(1)

    # Pass through the layers
    _embedded_1, _embedded_2 = embedding(input_1), embedding(input_2)
    _lstm_1, _lstm_2 = lstm(_embedded_1), lstm(_embedded_2)

    _concatenated = concatenate([_lstm_1, _lstm_2])
    _hidden_output = hidden(_concatenated)
    _output = output(_hidden_output)

    # Define the model
    model = tf.keras.Model(inputs=(input_1, input_2), outputs=_output)
    model.compile(loss='mean_absolute_error',
                  optimizer=tf.keras.optimizers.Adam(0.01))
    return model

# Entrenament del model amb embeddings de Word2Vec
model_word2vec = build_and_compile_model(word2vec_embeddings, input_length=max_length, hidden_size=hidden_size)
model_word2vec.fit([data_1, data_2], labels, epochs=10, batch_size=2)

# Avalua els resultats dels models
loss_word2vec = model_word2vec.evaluate([data_1, data_2], labels)

print(f'Word2Vec Embeddings - Loss: {loss_word2vec}')

### Prova

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Defineix funcions per calcular els embeddings de frases
def compute_mean_embedding(text, model, embedding_dim):
    """
    Agafa els vectors d'embeddings de cada paraula en una frase o document i 
    calcula la mitjana dels vectors per obtenir una única representació vectorial 
    per a la frase o document.
    """
    vectors = [model[word] for word in text if word in model]
    if vectors:
        mean_vector = np.mean(vectors, axis=0)
    else:
        mean_vector = np.zeros(embedding_dim)
    return mean_vector

def compute_weighted_mean_embedding(text, model, word2tfidf, embedding_dim):
    """
    Utilitza una ponderació per a cada vector de paraula, com ara la freqüència 
    inversa del document (TF-IDF), per calcular una mitjana ponderada dels vectors.
    """
    vectors = [model[word] * word2tfidf[word] for word in text if word in model and word in word2tfidf]
    if vectors:
        weighted_mean_vector = np.mean(vectors, axis=0)
    else:
        weighted_mean_vector = np.zeros(embedding_dim)
    return weighted_mean_vector

In [ ]:
# Exemple de corpus
corpus = ["Aquest és un text de exemple", "Aquí hi ha un altre text"]

# Prepara els vectors TF-IDF
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(corpus)
word2tfidf = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))

In [ ]:
# Exemple d'ús amb embeddings pre-entrenats (Word2Vec)
from gensim.models import KeyedVectors
import tensorflow as tf

# Carrega el model pre-entrenat (substitueix 'path_to_model' amb la ruta real)
# word2vec_model = KeyedVectors.load_word2vec_format('path_to_model', binary=True)

# Defineix la longitud de les seqüències i la dimensió dels embeddings
input_length = 10
embedding_dim = model_100MB.vector_size

# Prepara les dades d'entrenament
X_train1 = np.array([compute_mean_embedding(text.split(), model_100MB.wv, embedding_dim) for text in corpus])
X_train2 = np.array([compute_weighted_mean_embedding(text.split(), model_100MB.wv, word2tfidf, embedding_dim) for text in corpus])
y_train = np.random.rand(len(corpus), 1)

# Defineix el model de Keras (ajusta les dimensions segons la longitud de les seqüències i la dimensió dels embeddings)
def build_and_compile_model(input_dim, hidden_size=64):
    input_1 = tf.keras.Input(shape=(input_dim,))
    input_2 = tf.keras.Input(shape=(input_dim,))
    
    concatenate = tf.keras.layers.Concatenate(axis=-1)([input_1, input_2])
    hidden = tf.keras.layers.Dense(hidden_size, activation='relu')(concatenate)
    output = tf.keras.layers.Dense(1)(hidden)
    
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)
    model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(0.001))
    
    return model

# Construir i compilar el model
m = build_and_compile_model(input_dim=embedding_dim)

# Entrenar el model
m.fit([X_train1, X_train2], y_train, epochs=10, batch_size=32)

# Exemple de predicció
input_data1 = compute_mean_embedding("Aquest és un text de exemple".split(), model_100MB.wv, embedding_dim).reshape(1, -1)
input_data2 = compute_weighted_mean_embedding("Aquí hi ha un altre text".split(), model_100MB.wv, word2tfidf, embedding_dim).reshape(1, -1)

# Predir la similitud
y_pred = m.predict([input_data1, input_data2])
print(y_pred)

Epoch 1/10
1/1 [==============================] - 0s 334ms/step - loss: 0.7557
Epoch 2/10
1/1 [==============================] - 0s 3ms/step - loss: 0.1112
Epoch 3/10
1/1 [==============================] - 0s 2ms/step - loss: 0.4268
Epoch 4/10
1/1 [==============================] - 0s 5ms/step - loss: 0.5486
Epoch 5/10
1/1 [==============================] - 0s 3ms/step - loss: 0.4800
Epoch 6/10
1/1 [==============================] - 0s 5ms/step - loss: 0.3008
Epoch 7/10
1/1 [==============================] - 0s 3ms/step - loss: 0.0543
Epoch 8/10
1/1 [==============================] - 0s 4ms/step - loss: 0.2087
Epoch 9/10
1/1 [==============================] - 0s 2ms/step - loss: 0.2880
Epoch 10/10
1/1 [==============================] - 0s 55ms/step
[[0.10263795]]
